<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Persist, Deploy, and Score an SPSS Model to Predict Customer Churn</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook demonstrates how to deploy an SPSS model and score test data. 

You will use the **Telco Customer Churn** data set which contains anonymous data about customers of a fictitious telecommunication company. Use the details of this data set to predict customer churn which is critical to business as it is easier to retain existing customers rather than acquire new ones.

Some familiarity with Python is helpful. This notebook is compatible with CP4D 3.0 and Python 3.6.


## Learning goals

In this notebook you learn how to:

-  Save an SPSS model.
-  Create a batch deployment of the SPSS model.
-  Use the deployed model to score data.


## Contents

1.	[Setting up](#setup)
2.	[Save the SPSS model](#save)
3.  [Create a batch deployment (with deployment space only)](#deploy)
4.  [Score the model](#score)<br>
    4.1 [Create a Cloud Object Storage (COS) bucket](#cos)<br>
    4.2 [Create a connection](#connection)<br>
    4.3 [Create a connected data asset](#connectedAsset)<br>
    4.4 [Use the data to score the model](#predict)<br>
4.	[Summary](#summary)

To get started on CP4D 3.0, find documentation on installation and set up <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/cpd/overview/welcome.html" target="_blank" rel="noopener no referrer">here</a>.

## 1. Setting up <a id="setup"></a>

Import the `watson-machine-learning-client` module.
<div class="alert alert-block alert-info">
For more information about the <b>Watson Machine Learning Python client (V4)</b>, please refer to the <a href="https://wml-api-pyclient-dev-v4.mybluemix.net/" target="_blank" rel="noopener no referrer">Python client documentation</a>. If you're using the notebook within a project on your CP4D cluster, you do not need to install this package as it comes pre-installed with the notebooks. The installation code below is for demonstration but is non-executable at this stage.
</div>

In [1]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**Authenticate the Python client on CP4D.**

<div class="alert alert-block alert-info">To find your authentication information (your credentials) follow the steps provided here in the <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/ml-authentication-local.html" target="_blank" rel="noopener no referrer">Documentation.</a></div>

In [2]:
# Enter your credentials here.

from project_lib.utils import environment
url = environment.get_common_api_url()

import sys,os,os.path
token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
     "instance_id": "openshift",
     "token": token,
     "url": url,
     "version": "3.0.0"
}

Now, instantiate a `WatsonMachineLearningAPIClient` object.

In [3]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [4]:
client.version

'1.0.64'

<div class="alert alert-block alert-info">
You have a choice to either save the model in the <b>project</b> or the <b>deployment space</b>:<br><br>
    <li> If you're saving the model in your project, you have to set the default project using the python client.</li><br>
    <li>If you're saving the model in the deployment space, you have to obtain the space UID of the deployment space you've created. Then you'd use this to set the default space using the python client. From there you'll be able to deploy and score the model in your deployment space.</li></div>

### To set the default project, use the following code.

In [5]:
from project_lib import Project
project = Project.access()
project_id = project.get_metadata()["metadata"]["guid"]

client.set.default_project(project_id)

'SUCCESS'

### To set the default space, follow these steps.

<div class="alert alert-block alert-info">
You can create your own <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/ml-spaces_local.html" target="_blank" rel="noopener no referrer">deployment space</a> by selecting <b>Analytics deployments</b> under <b>Analyze</b> from the Navigation Menu on the top left of this page.</div>

Alternatively, you can create a deployment and obtain its UID using the code in the following cell. The cell is not executable cell at this stage, but you can enter the name of your space in the metadata and use it if needed.

In [6]:
# Obtain the UId of your space
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['guid'])

**Action:** Enter the name of your deployment space in the code below: `space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')`.

In [7]:
# Enter the name of your deployment space here:
space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')
print("Space UID = " + space_uid)

Space UID = 594b22d1-588c-4de0-9cbd-d98e6fa890e8


You can set the default space using the cell below.

In [8]:
client.set.default_space(space_uid)

Unsetting the project_id ...


'SUCCESS'

<a id="save"></a>
## 2. Save the SPSS model

Download the SPSS sample model from the <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener no referrer">Git repository</a>.

<div class="alert alert-block alert-info">
<b>Note:</b> You may need to install the <tt>wget</tt> package. To install the <tt>wget</tt> package, run the following command.</div>

In [9]:
!pip install --upgrade wget

Requirement already up-to-date: wget in /user-home/_global_/python-3 (3.2)


In [10]:
# Download the sample SPSS model.
import os
import wget

sample_dir = 'spss_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)

filename=os.path.join(sample_dir, 'customer-satisfaction-prediction.str')
if not os.path.isfile(filename):
    filename = wget.download('https://github.com/pmservice/wml-sample-models/raw/master/spss/customer-satisfaction-prediction/model/customer-satisfaction-prediction.str',\
                             out=sample_dir)
print(filename)

spss_sample_model/customer-satisfaction-prediction.str


Save the downloaded SPSS sample model as *SPSS model for Churn prediction*. First, you need to create the model metadata.

In [11]:
# Save the SPSS model to the WML repository.
# Model Metadata.
software_spec_uid = client.software_specifications.get_uid_by_name('spss-modeler_18.1')

meta_props={
    client.repository.ModelMetaNames.NAME: "SPSS model for Churn prediction",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    client.repository.ModelMetaNames.TYPE: "spss-modeler_18.1"
}

<div class="alert alert-block alert-info">To list the supported software specifications, run <tt>client.software_specifications.list()</tt>.<br>To find more information about the frameworks with their respective <b>Types</b> and <b>Software Specifications</b>, visit the <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/wmls/wmls-deploy-python-types.html" target="_blank" rel="noopener no referrer">documentation</a>.</div>

You can extract the model UID from the saved model details.

In [12]:
# Create the model artifact.
model_artifact = client.repository.store_model(filename, meta_props=meta_props)
model_uid = client.repository.get_model_uid(model_artifact)
print("Model UID = " + model_uid)

Model UID = 4a15c2ec-56a9-440b-a81c-5468e6b5f81b


Get the saved model metadata using the model UID.

In [13]:
# Details about the model.
model_details = client.repository.get_details(model_uid)
from pprint import pprint
pprint(model_details)

{'entity': {'content_status': {'state': 'persisted'},
            'name': 'SPSS model for Churn prediction',
            'software_spec': {'id': '5c3cad7e-507f-4b2a-a9a3-ab53a21dee8b'},
            'space': {'href': '/v4/spaces/594b22d1-588c-4de0-9cbd-d98e6fa890e8',
                      'id': '594b22d1-588c-4de0-9cbd-d98e6fa890e8'},
            'type': 'spss-modeler_18.1'},
 'metadata': {'created_at': '2020-05-12T04:04:21.002Z',
              'guid': '4a15c2ec-56a9-440b-a81c-5468e6b5f81b',
              'href': '/v4/models/4a15c2ec-56a9-440b-a81c-5468e6b5f81b?space_id=594b22d1-588c-4de0-9cbd-d98e6fa890e8',
              'id': '4a15c2ec-56a9-440b-a81c-5468e6b5f81b',
              'modified_at': '2020-05-12T04:04:24.002Z',
              'name': 'SPSS model for Churn prediction',
              'owner': '1000330999',
              'space_id': '594b22d1-588c-4de0-9cbd-d98e6fa890e8'}}


You can list all stored models using the `list_models` method.

In [14]:
# Display a list of all the models.
client.repository.list_models()

------------------------------------  -------------------------------  ------------------------  -----------------
GUID                                  NAME                             CREATED                   TYPE
4a15c2ec-56a9-440b-a81c-5468e6b5f81b  SPSS model for Churn prediction  2020-05-12T04:04:21.002Z  spss-modeler_18.1
90a7aef0-2d46-49ed-994d-c065512828f2  SPSS model for Churn prediction  2020-05-12T03:56:45.002Z  spss-modeler_18.1
81a55ede-932a-44d5-86be-ef48cbed778d  Tensorflow custom library        2020-05-12T03:33:45.002Z  tensorflow_1.15
------------------------------------  -------------------------------  ------------------------  -----------------


<div class="alert alert-block alert-info">
From the list of stored models, you can see that model is successfully saved.<br><br>
If you've set the default project, this means you've saved the model in your project. You can see the saved model in your project UI by clicking on your project name in the breadcrumb at the top of the application.<br><br>
If you've set the default space, this means that you've saved the model in your deployment space. You can view your model by selecting <b>Analytics Deployments</b> under <b>Analyze</b> from the Navigation Menu and clicking on your deployment space name.</div>

If you're using a deployment space, proceed to Section 3: [Create a batch deployment (with deployment space only)](#deploy). If not, you may skip to the [summary](#summary).

## 3. Create a batch deployment (with deployment space only) <a id="deploy"></a>

Use the code below to create a <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/deploy-batch-jobs.html" target="_blank" rel="noopener no referrer">batch deployment</a> using the model UID obtained in the previous section. Create the deployment metadata using the code below.

In [15]:
# Deployment metadata.
deploy_meta = {
    client.deployments.ConfigurationMetaNames.NAME: "Sample SPSS model deployment",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {"name": "S", "num_nodes": 1}
}

In [16]:
# Create the deployment.
deployment_details = client.deployments.create(model_uid, meta_props=deploy_meta)



#######################################################################################

Synchronous deployment creation for uid: '4a15c2ec-56a9-440b-a81c-5468e6b5f81b' started

#######################################################################################


ready.


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='02d47cec-562c-4f63-8d87-f47c55246775'
------------------------------------------------------------------------------------------------




Get the list of all deployments in the deployment space.

In [17]:
# List the deployments.
client.deployments.list()

------------------------------------  ------------------------------------  -----  ------------------------  -------------
GUID                                  NAME                                  STATE  CREATED                   ARTIFACT_TYPE
02d47cec-562c-4f63-8d87-f47c55246775  Sample SPSS model deployment          ready  2020-05-12T04:04:27.203Z  model
4de63cb0-974e-4a1f-8e6d-2ef18119528a  Sample SPSS model deployment          ready  2020-05-12T03:56:56.003Z  model
ec0c61f1-9134-48f4-a4e5-0feaa0efc60e  Tensorflow custom library deployment  ready  2020-05-12T03:33:51.111Z  model
------------------------------------  ------------------------------------  -----  ------------------------  -------------


<div class="alert alert-block alert-info">
From the list of deployed models, you can see that model was successfully deployed in the deployment space.</div>

Now, you can check details of your deployments.

In [18]:
# Deployment UID.
deployment_uid = client.deployments.get_uid(deployment_details)
print('Deployment uid = {}'.format(deployment_uid))

Deployment uid = 02d47cec-562c-4f63-8d87-f47c55246775


In [19]:
# Deployment details.
print(client.deployments.get_details(deployment_uid))

{'metadata': {'parent': {'href': ''}, 'name': 'Sample SPSS model deployment', 'guid': '02d47cec-562c-4f63-8d87-f47c55246775', 'description': '', 'id': '02d47cec-562c-4f63-8d87-f47c55246775', 'modified_at': '2020-05-12T04:04:27.203Z', 'created_at': '2020-05-12T04:04:27.203Z', 'href': '/v4/deployments/02d47cec-562c-4f63-8d87-f47c55246775', 'space_id': '594b22d1-588c-4de0-9cbd-d98e6fa890e8'}, 'entity': {'name': 'Sample SPSS model deployment', 'custom': {}, 'description': '', 'batch': {}, 'space': {'id': '594b22d1-588c-4de0-9cbd-d98e6fa890e8', 'href': '/v4/spaces/594b22d1-588c-4de0-9cbd-d98e6fa890e8'}, 'status': {'state': 'ready'}, 'asset': {'id': '4a15c2ec-56a9-440b-a81c-5468e6b5f81b', 'href': '/v4/models/4a15c2ec-56a9-440b-a81c-5468e6b5f81b?space_id=594b22d1-588c-4de0-9cbd-d98e6fa890e8'}, 'space_id': '594b22d1-588c-4de0-9cbd-d98e6fa890e8', 'hardware_spec': {'id': 'e7ed1d6c-2e89-42d7-aed5-863b972c1d2b', 'name': 'S', 'num_nodes': 1}}}


## Score the model <a id="score"></a>

In this section, you will go through the following steps.
- 4.1 [Create a Cloud Object Storage (COS) bucket](#cos)<br>
- 4.2 [Create a connection](#connection)<br>
- 4.3 [Create a connected data asset](#connectedAsset)<br>
- 4.4 [Use the data to score the model](#predict)<br>

### 4.1 Create a Cloud Object Storage (COS) bucket <a id="cos"></a>

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/cloud-object-storage" target="_blank" rel="noopener no referrer">Cloud Object Storage (COS)</a> instance (a lite plan is offered and information about how to order storage is <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml_dlaas_object_store.html?audience=wdp" target="_blank" rel="noopener no referrer">here</a>). 
- Create new credentials with HMAC: 
    - Go to your COS dashboard.
    - In the **Service credentials** tab, click **New Credential+**.
    - Add the inline configuration parameter: {"HMAC":true}, click **Add**. (For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml_dlaas_object_store.html#createcredentials" target="_blank" rel="noopener no referrer">HMAC</a>.)

    This configuration parameter adds the following section to the instance credentials, (for use later in this notebook):
    ```
      "cos_hmac_keys": {
            "access_key_id": "-------",
            "secret_access_key": "-------"
       }
    ```


In [20]:
# Import the boto library
import ibm_boto3
from ibm_botocore.client import Config
import os

Authenticate to COS and define the endpoint you will use.

1. Enter your COS credentials in the following cell. You can find these credentials in your COS instance dashboard under the **Service credentials** tab as described above.

2. Go to the Endpoint tab in the COS instance's dashboard to get the endpoint information, for example: `s3-api.us-geo.objectstorage.softlayer.net`. Enter it in the cell containing the variable `service_endpoint`.

In [21]:
# Enter your COS credentials.
cos_credentials = {
  "apikey": "***",
  "cos_hmac_keys": {
    "access_key_id": "***",
    "secret_access_key": "***"
  },
  "endpoints": "https://cos-service.bluemix.net/endpoints",
  "iam_apikey_description": "***",
  "iam_apikey_name": "***",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "***",
  "resource_instance_id": "***"
}

In [22]:
# Enter your Endpoint information.
service_endpoint = "***"

Create the Boto resource.

In [23]:
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint='https://iam.bluemix.net/oidc/token',
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

Create the buckets you will use to store training data and training results.

**Note:** Bucket names must be unique.

In [24]:
# Create a bucket
from uuid import uuid4
bucket = 'spss-scoring-payload' + str(uuid4())

if not cos.Bucket(bucket) in cos.buckets.all():
    print('Creating bucket "{}"...'.format(bucket))
    try:
        cos.create_bucket(Bucket=bucket)
    except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
        print('Error: {}.'.format(e.response['Error']['Message']))

Creating bucket "spss-scoring-payload87fe6044-68bc-48f3-9566-a1205de54688"...


Check that the bucket was created.

In [25]:
print([x.name for x in list(cos.buckets.all()) if x.name.startswith('spss-scoring-payload')])

['spss-scoring-payload87fe6044-68bc-48f3-9566-a1205de54688', 'spss-scoring-payload8b16d0f3-b67a-4ead-9a07-be77e0dd6ca3']


Download the scoring input from the <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener no referrer">Git repository</a>.

In [26]:
filename='scoreInput.csv'
if not os.path.isfile(filename):
    filename = wget.download('https://github.com/pmservice/wml-sample-models/raw/master/spss/customer-satisfaction-prediction/data/scoreInput.csv')
print(filename)

scoreInput.csv


Upload the data file to the created bucket.

In [27]:
with open(filename, "rb") as data:
    cos.Bucket(bucket).upload_fileobj(data, filename)
    print('{} is uploaded.'.format(filename))

scoreInput.csv is uploaded.


### 4.2 Create a connection <a id="connection"></a>

You'll use the COS credentials and endpoint information to create the connection metadata. You need to choose the <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/manage-data/conn_types.html" target="_blank" rel="noopener no referrer">connection type</a> as Cloud Object Storage as shown in the code below.

In [28]:
# Get the UID of the COS type connection.
data_source_type_id = client.connections.get_datasource_type_uid_by_name('cloudobjectstorage')

# create the metadata for the connector.
conn_meta_props = {
    client.connections.ConfigurationMetaNames.NAME: 'New Connection using the Python client',
    client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: data_source_type_id,
    client.connections.ConfigurationMetaNames.DESCRIPTION: 'Create a connection object using a COS instance',
    client.connections.ConfigurationMetaNames.PROPERTIES: {
        'access_key': cos_credentials['cos_hmac_keys']['access_key_id'],
        'iam_url': 'https://iam.cloud.ibm.com/identity/token',  # used when getting assets from this connection
        'secret_key': cos_credentials['cos_hmac_keys']['secret_access_key'],
        'url': service_endpoint
    }
}

In [29]:
conn_details = client.connections.create(meta_props=conn_meta_props)

Creating connections...
SUCCESS


Get the connection UID.

In [30]:
conn_uid = client.connections.get_uid(conn_details)
print('Connection UID: {}'.format(conn_uid))

Connection UID: debf3d7f-3ff9-4d46-99c2-5b801744c474


### 4.3 Create a connected data asset <a id="connectedAsset"></a>

Use the following code to save the payload as a .csv and add it as a data asset in your deployment space. You'll use the connection UID and point to where the file is located in the COS bucket in the data asset metadata.

In [31]:
# Create the metadata for the data asset.
file_location = bucket+'/'+filename

asset_meta_props = {
    client.data_assets.ConfigurationMetaNames.NAME: 'spss-scoring-payload.csv',
    client.data_assets.ConfigurationMetaNames.CONNECTION_ID: conn_uid,
    client.data_assets.ConfigurationMetaNames.DESCRIPTION: 'Connection using COS instance',
    # This is the data will be stored by WML.
    client.data_assets.ConfigurationMetaNames.DATA_CONTENT_NAME: file_location
}

In [32]:
# Store the asset and upload the data.
asset_details = client.data_assets.store(asset_meta_props)

Creating data asset...
SUCCESS


In [33]:
asset_details

{'metadata': {'space_id': '594b22d1-588c-4de0-9cbd-d98e6fa890e8',
  'guid': 'abab0af9-f0da-446d-a673-2d90fb647a7f',
  'href': '/v2/assets/abab0af9-f0da-446d-a673-2d90fb647a7f?space_id=594b22d1-588c-4de0-9cbd-d98e6fa890e8',
  'asset_type': 'data_asset',
  'created_at': '2020-05-12T04:04:44Z',
  'last_updated_at': '2020-05-12T04:04:44Z'},
 'entity': {'data_asset': {'mime_type': 'text/csv'}}}

You can see the connected data asset is stored in your deployment space.

In [34]:
client.data_assets.list()

------------------------  ----------  ----  ------------------------------------
NAME                      ASSET_TYPE  SIZE  ASSET_ID
spss-scoring-payload.csv  data_asset  0     5fe83069-007c-4119-a20c-23c12c7a9620
spss-scoring-payload.csv  data_asset  0     abab0af9-f0da-446d-a673-2d90fb647a7f
------------------------  ----------  ----  ------------------------------------


Retrieve the URL of the stored asset.

In [35]:
asset_href = client.data_assets.get_href(asset_details)

### 4.3 Use the data to score the model <a id="predict"></a>

Prepare the scoring payload with values to score against the deployed model.

In [36]:
# Prepare job payload.
job_payload_asset = {
    client.deployments.ScoringMetaNames.INPUT_DATA_REFERENCES: [{
        "type": "data_asset",
        "connection": {},
        "location": {
            "href": asset_href
        },
    }],
    client.deployments.ScoringMetaNames.OUTPUT_DATA_REFERENCE: {
            "type": "data_asset",
            "connection": {},
            "location": {
                "name": "churn_results_{}.csv".format(deployment_uid),
                "description": "results"
            }
        }
    }
pprint(job_payload_asset)

{'input_data_references': [{'connection': {},
                            'location': {'href': '/v2/assets/abab0af9-f0da-446d-a673-2d90fb647a7f?space_id=594b22d1-588c-4de0-9cbd-d98e6fa890e8'},
                            'type': 'data_asset'}],
 'output_data_reference': {'connection': {},
                           'location': {'description': 'results',
                                        'name': 'churn_results_02d47cec-562c-4f63-8d87-f47c55246775.csv'},
                           'type': 'data_asset'}}


Use the following method to run the scoring.

In [37]:
job = client.deployments.create_job(deployment_id=deployment_uid, meta_props=job_payload_asset)
job_uid = client.deployments.get_job_uid(job)
print('Job uid = {}'.format(job_uid))

Job uid = bdc70437-eb66-4cad-9d6c-d3a959a16493


In [38]:
def poll_async_job(client, job_uid):
    import time
    while True:
        job_status = client.deployments.get_job_status(job_uid)
        print(job_status)
        state = job_status['state']
        if state == 'completed' or 'fail' in state:
            return client.deployments.get_job_details(job_uid)
        time.sleep(5)

In [39]:
# Perform prediction.
job_details_asset = poll_async_job(client, job_uid)
pprint(job_details_asset)

{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'queued', 'running_at': '', 'completed_at': ''}
{'state': 'que

You can see your that the `churn results .csv` has been created as a data asset.

In [40]:
client.data_assets.list()

------------------------------------------------------  ----------  ----  ------------------------------------
NAME                                                    ASSET_TYPE  SIZE  ASSET_ID
spss-scoring-payload.csv                                data_asset  0     5fe83069-007c-4119-a20c-23c12c7a9620
spss-scoring-payload.csv                                data_asset  0     abab0af9-f0da-446d-a673-2d90fb647a7f
churn_results_02d47cec-562c-4f63-8d87-f47c55246775.csv  data_asset  382   717b6398-2939-442f-b423-0f9bba5a70e1
churn_results_4de63cb0-974e-4a1f-8e6d-2ef18119528a.csv  data_asset  382   e16620b5-9b59-4c03-8e71-13668ce9cd14
------------------------------------------------------  ----------  ----  ------------------------------------


Get the UID of the results .csv and download the data asset.

In [41]:
import re
results_asset = job_details_asset['entity']['scoring']['output_data_reference']['location']['href']
results_uid = re.split('[?/]', results_asset)[3]
results_uid

'717b6398-2939-442f-b423-0f9bba5a70e1'

In [42]:
client.data_assets.get_details(results_uid)

{'metadata': {'space_id': '594b22d1-588c-4de0-9cbd-d98e6fa890e8',
  'guid': '717b6398-2939-442f-b423-0f9bba5a70e1',
  'href': '/v2/assets/717b6398-2939-442f-b423-0f9bba5a70e1?space_id=594b22d1-588c-4de0-9cbd-d98e6fa890e8',
  'asset_type': 'data_asset',
  'created_at': '2020-05-12T04:09:32Z',
  'last_updated_at': '2020-05-12T04:09:33Z'},
 'entity': {'data_asset': {'mime_type': 'application/csv', 'dataset': True}}}

You can see the values predicted by the model here.

In [43]:
client.data_assets.download(results_uid, "scoring_results.csv")

Successfully saved data asset content to file: 'scoring_results.csv'


'/home/wsuser/work/scoring_results.csv'

In [44]:
import pandas as pd
pd.read_csv("scoring_results.csv")

,customerID,Churn,Predicted Churn,Probability of Churn
0,9237-HQITU,Yes,Yes,0.882983
1,3638-WEABW,No,No,0.052631
2,8665-UTDHZ,Yes,No,0.174110
3,8773-HHUOZ,Yes,No,0.484323
4,4080-IIARD,No,No,0.092014
5,6575-SUVOI,No,No,0.092092
6,7495-OOKFY,Yes,Yes,0.972150
7,0731-EBJQB,No,No,0.090598
8,1891-QRQSA,No,No,0.092103
9,5919-TMRGD,Yes,Yes,0.894228


The first 2 customers are said to predicted to have churned and not churned respectively.

You can delete the 2 created data assets, as well as the connection, deployment, and model using the code below.

In [45]:
# client.data_assets.delete(client.data_assets.get_uid(asset_details))
# client.data_assets.delete(results_uid)

In [46]:
#client.connections.delete(conn_uid)

In [47]:
# client.deployments.delete(deployment_uid)

In [48]:
# client.repository.delete(model_uid)

## 5. Summary <a id="summary"></a>

You successfully completed this notebook! 

You learned how to use Watson Machine Learning to create and deploy an SPSS model. You learned how to create a batch deployment, a connection, and a connected data asset using Watson Machine Learning.

### Resources <a id="resources"></a>

To learn more about configurations used in this notebook or more sample notebooks, tutorials, documentation, how-tos, and blog posts, check out these links:

<div class="alert alert-block alert-success"><a id="resources"></a>

<h4>IBM documentation</h4>
 <ul>
 <li> <a href="https://wml-api-pyclient-dev-v4.mybluemix.net" target="_blank" rel="noopener no referrer">watson-machine-learning</a></li> 
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/cpd/overview/welcome.html" target="_blank" rel="noopener noreferrer">CP4D 3.0</a></li>
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/deploy-batch-jobs.html" target="_blank" rel="noopener no referrer">Batch Deployments</a></li>
     <ul>
         <li> <a href="https://www.ibm.com/support/knowledgecenter/SSHGWL_3.0.0/wsj/wmls/wmls-deploy-python.html#deploy-batch" target="_blank" rel="noopener no referrer">Batch Deployments with the Python client</a></li>
         <li> <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/deploy-batch-details.html" target="_blank" rel="noopener no referrer">Batch Deployment jobs</a></li>
    </ul>
 </ul>

<h4> IBM Samples</h4>
<br>
 <li> <a href="https://github.com/IBMDataScience/sample-notebooks" target="_blank" rel="noopener noreferrer">Sample notebooks</a></li>
 
<h4> Others</h4>
<br>
 <li> <a href="https://www.python.org" target="_blank" rel="noopener noreferrer">Official Python website</a></li>
 </div>

### Authors

**Lukasz Cmielowski**, Ph.D., is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increase the clients' ability to turn data into actionable knowledge.  
**Jihyoung Kim**, Ph.D., is a Data Scientist at IBM who strives to make data science easy for everyone through Watson Studio.

<hr>
Copyright © 2017-2020 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>